In [62]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.pipeline import make_pipeline, make_union
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from tpot import *
from xgboost import *
from sklearn.ensemble import *
from sklearn.svm import *
import warnings
warnings.filterwarnings('ignore')

# DATASET : Owner median < 20 k

In [63]:
df = pd.read_csv('../raw_data/data_final_indé_small.csv')

In [64]:
df.head(2)

,steam_appid,name,top_5_tags,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,topic_17,topic_18,topic_19,english,price,days_on_steam,is_a_remake,has_a_website,owner_estimated,score_descriptif
0,4290,RACE: Caterham Expansion,racing simulation relaxing realistic real_time...,0.015822,0.015822,0.015822,0.015822,0.015822,0.015822,0.015822,...,0.015822,0.015822,0.015822,1,3.99,4331.0,1,1,1248.0,4
1,4900,Zen of Sudoku,casual puzzle 1980s relaxing realistic,0.015666,0.307211,0.015666,0.015666,0.015666,0.015666,0.015666,...,0.015666,0.015666,0.410797,1,0.00,4521.0,1,1,20000.0,3


In [65]:
df.columns

Index(['steam_appid', 'name', 'top_5_tags', 'topic_0', 'topic_1', 'topic_2',
       'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8',
       'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14',
       'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19', 'english',
       'price', 'days_on_steam', 'is_a_remake', 'has_a_website',
       'owner_estimated', 'score_descriptif'],
      dtype='object')

In [66]:
#df.corr().style.background_gradient(cmap='coolwarm')

In [67]:
X = df.drop(columns=['steam_appid', 'name', 'top_5_tags','score_descriptif','owner_estimated'])

In [68]:
X.shape

(18108, 25)

In [69]:
y = df['owner_estimated']

In [70]:
y.shape

(18108,)

#### Train Test Split


In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [72]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12675, 25), (5433, 25), (12675,), (5433,))

In [73]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

ensemble.fit(X_train, y_train)

print(ensemble.score(X_test,y_test))

0.2895967416594285


In [44]:
# instanciate TPOTRegressor
tpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=2)
# process autoML with TPOT
tpot.fit(X_train, y_train)

# print score
print(tpot.score(X_test, y_test))

Optimization Progress:   0%|          | 0/250 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.27243924191829255

Generation 2 - Current best internal CV score: 0.27243924191829255

Generation 3 - Current best internal CV score: 0.27376567777909677

Generation 4 - Current best internal CV score: 0.27376567777909677

Best pipeline: XGBRegressor(LinearSVR(input_matrix, C=20.0, dual=False, epsilon=1.0, loss=squared_epsilon_insensitive, tol=1e-05), learning_rate=0.1, max_depth=5, min_child_weight=14, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.8500000000000001, verbosity=0)
0.29177572342287117


In [74]:
X_train.columns

Index(['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
       'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11',
       'topic_12', 'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17',
       'topic_18', 'topic_19', 'english', 'price', 'days_on_steam',
       'is_a_remake', 'has_a_website'],
      dtype='object')

## TEST avec une nouvelle feature : Ratio 

In [87]:
dfa= pd.read_csv('../raw_data/data_final_indé_small2.csv')

In [88]:
Xa = dfa.drop(columns=['steam_appid', 'name', 'top_5_tags','score_descriptif','owner_estimated','sells_per_days'])

In [89]:
ya = dfa['owner_estimated']

#### Train test split

In [90]:
Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa, ya, test_size = 0.3, random_state = 0)

In [52]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

scores = cross_val_score(ensemble, Xa, ya, cv=5) # Cross validate

print(scores)

/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12

[0.29142866 0.3070158  0.45637449 0.48873325 0.51606764]


In [53]:
scores.mean()

0.4119239698891072

In [97]:
# instanciate TPOTRegressor
tpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=5)
# process autoML with TPOT
tpot.fit(Xa_train, ya_train)

# print score
print(tpot.score(Xa_test, ya_test))

Optimization Progress:   0%|          | 0/250 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.4348205642582874

Generation 2 - Current best internal CV score: 0.43810748162881674

Generation 3 - Current best internal CV score: 0.43810748162881674

Generation 4 - Current best internal CV score: 0.43861958298278647

Best pipeline: RandomForestRegressor(RobustScaler(input_matrix), bootstrap=True, max_features=0.9500000000000001, min_samples_leaf=1, min_samples_split=20, n_estimators=100)
0.44888206290757127


/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12

## TEST modèle en enlevant les jeux < 1 €


In [91]:
dfa = dfa[dfa['price']>= 1]

In [92]:
Xa_ = dfa.drop(columns=['steam_appid', 'name', 'top_5_tags','score_descriptif','owner_estimated','','sells_per_days'])

In [93]:
ya_ = dfa['owner_estimated']

In [94]:
Xa__train, Xa__test, ya__train, ya__test = train_test_split(Xa_, ya_, test_size = 0.3, random_state = 0)

In [95]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

scores = cross_val_score(ensemble, Xa_, ya_, cv=5) # Cross validate

print(scores.mean())

/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12

0.402545218410001


In [99]:
ensemble.fit(Xa__train, ya__train)

StackingRegressor(estimators=[('lr',
                               RandomForestRegressor(max_features=0.4,
                                                     min_samples_leaf=14,
                                                     min_samples_split=14)),
                              ('svr', LinearSVR(random_state=42)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_...
                                            learning_rate=0.1,
                               

In [102]:
ow_df = ensemble.predict(Xa__test)

In [103]:
o_df = pd.DataFrame(ow_df)

In [104]:
o_df

,0
0,5446.651532
1,7678.388311
2,2013.884473
3,926.763287
4,3091.082033
...,...
4431,2046.351629
4432,1346.290096
4433,12671.608340
4434,8315.913311


#### Test tpot

In [96]:
# instanciate TPOTRegressor
tpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=5)
# process autoML with TPOT
tpot.fit(Xa__train, ya__train)

# print score
print(tpot.score(Xa__test, ya__test))

Optimization Progress:   0%|          | 0/250 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.4821433933167252

Generation 2 - Current best internal CV score: 0.4821433933167252

Generation 3 - Current best internal CV score: 0.4821433933167252

Generation 4 - Current best internal CV score: 0.4821433933167252

Best pipeline: XGBRegressor(input_matrix, learning_rate=0.5, max_depth=3, min_child_weight=17, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.9000000000000001, verbosity=0)
0.47440452171201763
